### Caplets and floorlet
A caplet is similar to a European call option on the interest rate, $ r_t $.

Usually "settled in arrears" but they may also be settled in advance - which means payoff at t will be paid at t+1 (we'll hence need to discount our payments at the last period.)

If maturity is τ and strike is c, then payoff of a caplet (settled in arrears) at
time τ is max(0, $ r_{t-1} $ − c) so the caplet is a call option on the short rate prevailing at time τ − 1, settled at time τ .

A floorlet is the same as a caplet except the payoff is max(0, c − $r_{τ−1}$).

A cap consists of a sequence of caplets all of which have the same strike.
A floor consists of a sequence of floorlets all of which have the same strike.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from math import exp, sqrt, pow
import pandas as pd

In [48]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

In [2]:
def short_rate(u, d, n, r):
    shortRateTree = np.zeros((n+1, n+1))  
    shortRateTree[0,0] = r
    for i in range(1,n+1):
        shortRateTree[0,i] = shortRateTree[0, i-1]*u
        for j in range(1,n+1):
            shortRateTree[j,i] = shortRateTree[j-1, i-1]*d
            
    return shortRateTree

In [3]:
print(short_rate(1.25, 0.9, 5, 6))
#This is a rather unrealistic situation for we can see interest rates rising upto 18% or more which does not happen in the economy, but
#for the sake of an argument

[[ 6.          7.5         9.375      11.71875    14.6484375  18.31054688]
 [ 0.          5.4         6.75        8.4375     10.546875   13.18359375]
 [ 0.          0.          4.86        6.075       7.59375     9.4921875 ]
 [ 0.          0.          0.          4.374       5.4675      6.834375  ]
 [ 0.          0.          0.          0.          3.9366      4.92075   ]
 [ 0.          0.          0.          0.          0.          3.54294   ]]


In [11]:
def price_caplet(u, d, n, r, c):
    '''Caplet has a strike price c'''
    # pay off at the final time period, ie, n is max(c-r_n , 0) which will be paid at n+1 hence discounted by (1+r_n)
    shortRateTree = short_rate(u, d, n, r)
    capletPrice = np.zeros((n+1, n+1))
    q1 = 0.5
    q2 = 1 - q1
    for i in range(n+1):
        capletPrice[i, n] = (max(0, shortRateTree[i,n]/100 - c/100))/(1+shortRateTree[i,n]/100)
    i = n-1
    while i+1:
        for j in range(i+1):
            capletPrice[j, i] = (q2 * capletPrice[j, i+1] + q1 * capletPrice[j+1, i+1])/(1+shortRateTree[j,i]/100)
        i-=1
    return capletPrice
    

In [12]:
print(price_caplet(1.25, 0.9, 5, 6, 2))

[[0.04204522 0.05150267 0.06367244 0.08004766 0.10321618 0.13786215]
 [0.         0.03763321 0.0470583  0.0592358  0.07564031 0.09880932]
 [0.         0.         0.0322725  0.04123367 0.05282733 0.06842669]
 [0.         0.         0.         0.02644821 0.03464991 0.04525112]
 [0.         0.         0.         0.         0.02056019 0.02783768]
 [0.         0.         0.         0.         0.         0.01490145]]


### Interest rate swaps

In [24]:
def price_swaps(u, d, n, r, c):
    '''swap price is +-(r[t]-c) where c is the strike price, It is the constant payment done in every time period. 
    The final price is discounted since it is payed in arears as well'''
    shortRateTree = short_rate(u, d, n, r )
    swapPrice = np.zeros((n+1, n+1))
    q1 = 0.5
    q2 = 1 - q1
    for i in range(n+1):
        swapPrice[i, n] = (shortRateTree[i,n]/100 - c/100)/(1+shortRateTree[i,n]/100)
    i = n-1
    while i+1:
        for j in range(i+1):
            swapPrice[j, i] = ((shortRateTree[j,i]/100 - c/100) + q2 * swapPrice[j, i+1] + q1 * swapPrice[j+1, i+1])/(1+shortRateTree[j,i]/100)
        i-=1
    return swapPrice
    

In [25]:
price_swaps(1.25, 0.9, 5, 6, 5)

array([[0.0990, 0.1403, 0.1686, 0.1793, 0.1648, 0.1125],
       [0.0000, 0.0496, 0.0829, 0.1021, 0.1014, 0.0723],
       [0.0000, 0.0000, 0.0137, 0.0400, 0.0512, 0.0410],
       [0.0000, 0.0000, 0.0000, -0.0085, 0.0122, 0.0172],
       [0.0000, 0.0000, 0.0000, 0.0000, -0.0174, -0.0008],
       [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, -0.0141]])

### SWAPSONS
Swapsons are options on swaps.
We create the swap payoff lattice
Then we create an option lattice on top of it, based on the time period of the option and strike of the option (which is different from 
strike of the swap)

In [29]:
def price_swapsons(u, d, n, r, c, N, K):
    '''N is the maturity of the swapson and K is the strike of the option'''
    shortRateTree = short_rate(u, d, n, r )
    swapPrice = price_swaps(u, d, n, r, c)
    swapsonPrice = np.zeros((N+1, N+1))
    q1 = 0.5
    q2 = 1 - q1
    for i in range(N+1):
        swapsonPrice[i, N] = max(0, swapPrice[i,N]-K/100)
    i = N-1
    while i+1:
        for j in range(i+1):
            swapsonPrice[j, i] = (q2 * swapsonPrice[j, i+1] + q1 * swapsonPrice[j+1, i+1])/(1+shortRateTree[j,i]/100)
        i-=1
    return swapsonPrice
    

In [30]:
price_swapsons(1.25, 0.9, 5, 6, 5, 3, 0)

array([[0.0620, 0.0908, 0.1286, 0.1793],
       [0.0000, 0.0406, 0.0665, 0.1021],
       [0.0000, 0.0000, 0.0191, 0.0400],
       [0.0000, 0.0000, 0.0000, 0.0000]])

### Forward equations

In [40]:
def forward_equations(u, d, n, r):
    forwardTree = np.zeros((n+1, n+1))
    shortRate = short_rate(u, d, n, r)
    print(shortRate)
    forwardTree[0,0]=1
    for i in range(1, n+1):
        for j in range(i+1):
            if j==0:
                forwardTree[j, i] = 0.5*(forwardTree[j,i-1]/(1+shortRate[j, i-1]/100))
            else:
                forwardTree[j, i] = 0.5*(forwardTree[j,i-1]/(1+shortRate[j, i-1]/100)) + 0.5*(forwardTree[j-1,i-1]/(1+shortRate[j-1, i-1]/100))
            
    return forwardTree

In [43]:
forward_equations(1.25, 0.9, 5, 6)

[[6.0000 7.5000 9.3750 11.7188 14.6484 18.3105]
 [0.0000 5.4000 6.7500 8.4375 10.5469 13.1836]
 [0.0000 0.0000 4.8600 6.0750 7.5938 9.4922]
 [0.0000 0.0000 0.0000 4.3740 5.4675 6.8344]
 [0.0000 0.0000 0.0000 0.0000 3.9366 4.9207]
 [0.0000 0.0000 0.0000 0.0000 0.0000 3.5429]]


array([[1.0000, 0.4717, 0.2194, 0.1003, 0.0449, 0.0196],
       [0.0000, 0.4717, 0.4432, 0.3079, 0.1868, 0.1041],
       [0.0000, 0.0000, 0.2238, 0.3143, 0.2901, 0.2193],
       [0.0000, 0.0000, 0.0000, 0.1067, 0.1992, 0.2293],
       [0.0000, 0.0000, 0.0000, 0.0000, 0.0511, 0.1190],
       [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0246]])

In [53]:
def price_swap_forward_equation(u, d, n, r, c):
    shortRate = short_rate(u, d, n, r)
    forward = forward_equations(u, d, n, r)
    price = 0
    for i in range(1, n):
        for j in range(i+1):
            price += forward[j,i]*((c/100 - shortRate[j,i]/100)/(1+shortRate[j,i]/100))
    print(price)
    return price    

In [54]:
price_swap_forward_equation(1.25, 0.9, 3, 6, 7) * 1000000

[[6.00 7.50 9.38 11.72]
 [0.00 5.40 6.75 8.44]
 [0.00 0.00 4.86 6.08]
 [0.00 0.00 0.00 4.37]]
0.005807056684766097


5807.056684766097

In [57]:
price_swap_forward_equation(1.1, 0.9, 11, 5, 4.5)*1000000

[[5.00 5.50 6.05 6.66 7.32 8.05 8.86 9.74 10.72 11.79 12.97 14.27]
 [0.00 4.50 4.95 5.45 5.99 6.59 7.25 7.97 8.77 9.65 10.61 11.67]
 [0.00 0.00 4.05 4.46 4.90 5.39 5.93 6.52 7.17 7.89 8.68 9.55]
 [0.00 0.00 0.00 3.65 4.01 4.41 4.85 5.34 5.87 6.46 7.10 7.81]
 [0.00 0.00 0.00 0.00 3.28 3.61 3.97 4.37 4.80 5.28 5.81 6.39]
 [0.00 0.00 0.00 0.00 0.00 2.95 3.25 3.57 3.93 4.32 4.75 5.23]
 [0.00 0.00 0.00 0.00 0.00 0.00 2.66 2.92 3.22 3.54 3.89 4.28]
 [0.00 0.00 0.00 0.00 0.00 0.00 0.00 2.39 2.63 2.89 3.18 3.50]
 [0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 2.15 2.37 2.60 2.86]
 [0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 1.94 2.13 2.34]
 [0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 1.74 1.92]
 [0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 1.57]]
-0.03337424206216373


-33374.24206216373

In [33]:
#6.
price_swapsons(1.1, 0.9, 10, 5, 4.5, 5, 0)[0,0] * 1000000

26311.07949019225